In [1]:
import pandas as pd
from pathlib import Path

data_directory = Path('../data')

file_qualifying = data_directory / 'qualifying_results_2018_to_2025.csv'
file_races = data_directory / 'race_results_2018_to_2025.csv'
file_practice = data_directory / 'practice_laps_2018_to_2025.csv'
file_weather = data_directory / 'weather_data_2018_to_2025.csv'
file_schedule = data_directory / 'schedule_data_2018_to_2025.csv'
try:
    df_qualifying = pd.read_csv(file_qualifying)
    df_races = pd.read_csv(file_races)
    df_practice = pd.read_csv(file_practice, low_memory=False)
    df_weather = pd.read_csv(file_weather)
    df_schedule = pd.read_csv(file_schedule)
    print("Tất cả các file đã được tải thành công! ✅")

except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file tại '{e.filename}'. Vui lòng kiểm tra lại tên file và đường dẫn.")

Tất cả các file đã được tải thành công! ✅


In [2]:
df_weather_processed = df_weather[['Year', 'RaceName', 'Rainfall']].copy()

# Chuyển đổi cột 'Rainfall' thành dạng số (True=1, False=0)
df_weather_processed['Rainfall'] = df_weather_processed['Rainfall'].astype(int)

# Tổng hợp: Nếu có bất kỳ thời điểm nào ghi nhận có mưa (Rainfall > 0) trong suốt cuộc đua,
# chúng ta sẽ đánh dấu cả cuộc đua đó là "Có Mưa".
df_weather_agg = df_weather_processed.groupby(['Year', 'RaceName'])['Rainfall'].max().reset_index()

print("Xử lý thời tiết hoàn tất. Dữ liệu sau khi tổng hợp:")
print(df_weather_agg.head())

Xử lý thời tiết hoàn tất. Dữ liệu sau khi tổng hợp:
   Year               RaceName  Rainfall
0  2018   Abu Dhabi Grand Prix         1
1  2018  Australian Grand Prix         1
2  2018    Austrian Grand Prix         0
3  2018  Azerbaijan Grand Prix         0
4  2018     Bahrain Grand Prix         0


In [3]:
# --- Xử lý dữ liệu đua thử (phiên bản đã lọc các tay đua dự bị) ---

# 1. TẠO DANH SÁCH TAY ĐUA CHÍNH THỨC TỪ DF_RACES
# Danh sách này sẽ được dùng làm bộ lọc
official_drivers = df_races[['Year', 'RaceName', 'DriverNumber']].drop_duplicates()

# 2. CHUẨN BỊ DỮ LIỆU ĐUA THỬ
df_practice['LapTime_seconds'] = pd.to_timedelta(df_practice['LapTime'], errors='coerce').dt.total_seconds()
df_practice_valid = df_practice.dropna(subset=['LapTime_seconds'])

# 3. LỌC DỮ LIỆU ĐUA THỬ ĐỂ CHỈ GIỮ LẠI TAY ĐUA CHÍNH THỨC
# Dùng 'inner' merge để loại bỏ các tay đua dự bị không có trong 'official_drivers'
df_practice_filtered = pd.merge(df_practice_valid, official_drivers, on=['Year', 'RaceName', 'DriverNumber'], how='inner')

# 4. TỔNG HỢP DỮ LIỆU TRÊN BỘ DỮ LIỆU ĐÃ LỌC
practice_agg_dict = {
    'LapTime_seconds': ['median', 'min', 'std']
}
df_practice_agg = df_practice_filtered.groupby(['Year', 'RaceName', 'DriverNumber']).agg(practice_agg_dict).reset_index()

# 5. ĐỔI TÊN CỘT
df_practice_agg.columns = ['Year', 'RaceName', 'DriverNumber', 'PracticeMedianLapTime', 'PracticeFastestLapTime', 'PracticeLapTimeStdDev']

print("✅ Xử lý dữ liệu đua thử (đã lọc) thành công!")
print(df_practice_agg.head())

✅ Xử lý dữ liệu đua thử (đã lọc) thành công!
   Year              RaceName  DriverNumber  PracticeMedianLapTime  \
0  2018  Abu Dhabi Grand Prix             2               113.9500   
1  2018  Abu Dhabi Grand Prix             3               104.0840   
2  2018  Abu Dhabi Grand Prix             5               105.3075   
3  2018  Abu Dhabi Grand Prix             7               104.7635   
4  2018  Abu Dhabi Grand Prix             8               106.5530   

   PracticeFastestLapTime  PracticeLapTimeStdDev  
0                  99.938              17.829566  
1                  97.428              13.274270  
2                  97.569              17.608850  
3                  97.461              15.311805  
4                  98.060              14.798264  


In [4]:
columns_to_keep = [
    'Year',
    'RaceName',
    'DriverNumber',
    'TeamName',
    'GridPosition', # Lấy vị trí xuất phát chính thức từ đây [cite: 51, 52]
    'Position',      # Đây là vị trí về đích (target) [cite: 51]
    'Points'
]

# 2. Tạo DataFrame đã hợp nhất và đổi tên cột mục tiêu cho rõ ràng.
df_merged = df_races[columns_to_keep].copy()
df_merged.rename(columns={'Position': 'FinalPosition'}, inplace=True)

# 3. Loại bỏ các cuộc đua không có thông tin GridPosition hợp lệ (ví dụ: các phiên Sprint Shootout)
df_merged.dropna(subset=['GridPosition'], inplace=True)

# 4. Chuyển đổi GridPosition sang kiểu số nguyên
df_merged['GridPosition'] = df_merged['GridPosition'].astype(int)


print("Xây dựng DataFrame nền tảng thành công!")
print(f"Kích thước của DataFrame sau bước này: {df_merged.shape}")
print("\n5 hàng đầu của DataFrame nền tảng:")
print(df_merged.head())

Xây dựng DataFrame nền tảng thành công!
Kích thước của DataFrame sau bước này: (3335, 7)

5 hàng đầu của DataFrame nền tảng:
   Year               RaceName  DriverNumber         TeamName  GridPosition  \
0  2018  Australian Grand Prix             5          Ferrari             3   
1  2018  Australian Grand Prix            44         Mercedes             1   
2  2018  Australian Grand Prix             7          Ferrari             2   
3  2018  Australian Grand Prix             3  Red Bull Racing             8   
4  2018  Australian Grand Prix            14          McLaren            10   

   FinalPosition  Points  
0            1.0    25.0  
1            2.0    18.0  
2            3.0    15.0  
3            4.0    12.0  
4            5.0    10.0  


In [5]:
df_merged = pd.merge(df_merged, df_schedule, on=['Year', 'RaceName'], how='left')
# Chuyển cột 'EventDate' sang định dạng datetime để sắp xếp chính xác
df_merged['EventDate'] = pd.to_datetime(df_merged['EventDate'])

# Sắp xếp DataFrame theo Năm, sau đó theo Ngày diễn ra cuộc đua
df_merged = df_merged.sort_values(by=['Year', 'EventDate'])

# Reset lại index của DataFrame cho gọn gàng
df_merged.reset_index(drop=True, inplace=True)

print("\nDữ liệu cuối cùng sau khi đã sắp xếp:")
print(df_merged.head())


Dữ liệu cuối cùng sau khi đã sắp xếp:
   Year               RaceName  DriverNumber         TeamName  GridPosition  \
0  2018  Australian Grand Prix             5          Ferrari             3   
1  2018  Australian Grand Prix            44         Mercedes             1   
2  2018  Australian Grand Prix             7          Ferrari             2   
3  2018  Australian Grand Prix             3  Red Bull Racing             8   
4  2018  Australian Grand Prix            14          McLaren            10   

   FinalPosition  Points  RoundNumber  EventDate  
0            1.0    25.0            1 2018-03-25  
1            2.0    18.0            1 2018-03-25  
2            3.0    15.0            1 2018-03-25  
3            4.0    12.0            1 2018-03-25  
4            5.0    10.0            1 2018-03-25  


In [6]:
# --- TẠO FEATURE CHAMPIONSHIP STANDINGS (PHIÊN BẢN CUỐI CÙNG) ---

# Đảm bảo DataFrame đã được sắp xếp
df_merged.sort_values(by=['Year', 'RoundNumber'], inplace=True)
df_merged.reset_index(drop=True, inplace=True)

# Khởi tạo các cột mới
df_merged['DriverPoints_BeforeRace'] = 0
df_merged['ConstructorPoints_BeforeRace'] = 0

print("Bắt đầu tính toán điểm số (phiên bản cuối cùng)...")

# Lặp qua từng mùa giải
for year in df_merged['Year'].unique():
    # Reset điểm số ở đầu mỗi mùa giải
    season_driver_points = {}
    season_constructor_points = {}

    # Lặp qua từng chặng đua trong mùa giải đó
    for round_num in df_merged[df_merged['Year'] == year]['RoundNumber'].unique():

        # --- BƯỚC 1: GÁN ĐIỂM SỐ CỦA CHẶNG TRƯỚC ---
        # Lấy index của tất cả các tay đua trong chặng hiện tại
        current_race_indices = df_merged[(df_merged['Year'] == year) & (df_merged['RoundNumber'] == round_num)].index

        # Với mỗi tay đua, gán điểm số đã được tính từ các chặng TRƯỚC ĐÓ
        for index in current_race_indices:
            driver = df_merged.at[index, 'DriverNumber']
            team = df_merged.at[index, 'TeamName']

            df_merged.at[index, 'DriverPoints_BeforeRace'] = season_driver_points.get(driver, 0)
            df_merged.at[index, 'ConstructorPoints_BeforeRace'] = season_constructor_points.get(team, 0)

        # --- BƯỚC 2: CẬP NHẬT ĐIỂM SỐ CHO CHẶNG TIẾP THEO ---
        # Lấy kết quả của chặng hiện tại để cộng dồn điểm
        current_race_results = df_merged.loc[current_race_indices]

        for index, row in current_race_results.iterrows():
            driver = row['DriverNumber']
            team = row['TeamName']
            points_earned = row['Points']

            season_driver_points[driver] = season_driver_points.get(driver, 0) + points_earned
            season_constructor_points[team] = season_constructor_points.get(team, 0) + points_earned

# --- Chuyển đổi từ điểm số sang thứ hạng (Rank) ---
df_merged['DriverChampionshipStanding'] = df_merged.groupby('Year')['DriverPoints_BeforeRace'].rank(method='dense', ascending=False)
df_merged['ConstructorChampionshipStanding'] = df_merged.groupby('Year')['ConstructorPoints_BeforeRace'].rank(method='dense', ascending=False)


print("✅ Hoàn thành! Các feature về thứ hạng đã được tạo lại chính xác.")

# In ra kết quả để kiểm tra lại chặng đầu tiên của một mùa giải
print("\nKiểm tra kết quả chặng đầu tiên của mùa giải 2019:")
check_df = df_merged[(df_merged['RaceName'] == 'Australian Grand Prix') & (df_merged['Year'] == 2019)]
print(check_df[['DriverNumber', 'TeamName', 'Points', 'DriverPoints_BeforeRace', 'ConstructorPoints_BeforeRace']])

Bắt đầu tính toán điểm số (phiên bản cuối cùng)...


C:\Users\bachng\AppData\Local\Temp\ipykernel_9424\252801876.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '196.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_merged.at[index, 'DriverPoints_BeforeRace'] = season_driver_points.get(driver, 0)
C:\Users\bachng\AppData\Local\Temp\ipykernel_9424\252801876.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '300.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_merged.at[index, 'ConstructorPoints_BeforeRace'] = season_constructor_points.get(team, 0)


✅ Hoàn thành! Các feature về thứ hạng đã được tạo lại chính xác.

Kiểm tra kết quả chặng đầu tiên của mùa giải 2019:
     DriverNumber           TeamName  Points  DriverPoints_BeforeRace  \
420            77           Mercedes    26.0                      0.0   
421            44           Mercedes    18.0                      0.0   
422            33    Red Bull Racing    15.0                      0.0   
423             5            Ferrari    12.0                      0.0   
424            16            Ferrari    10.0                      0.0   
425            20       Haas F1 Team     8.0                      0.0   
426            27            Renault     6.0                      0.0   
427             7  Alfa Romeo Racing     4.0                      0.0   
428            18       Racing Point     2.0                      0.0   
429            26         Toro Rosso     1.0                      0.0   
430            10    Red Bull Racing     0.0                      0.0   
431    

In [7]:
print("\nThêm dữ liệu đua thử đã xử lý...")
# Merge với dữ liệu đua thử đã tổng hợp
df_merged = pd.merge(df_merged, df_practice_agg, on=['Year', 'RaceName', 'DriverNumber'], how='left')
print(f"Kích thước sau khi merge lần 2: {df_merged.shape}")


print("\nThêm dữ liệu thời tiết đã xử lý...")
# Merge với dữ liệu thời tiết đã tổng hợp
df_merged = pd.merge(df_merged, df_weather_agg, on=['Year', 'RaceName'], how='left')
print(f"Kích thước sau khi merge lần 3: {df_merged.shape}")


Thêm dữ liệu đua thử đã xử lý...
Kích thước sau khi merge lần 2: (3335, 16)

Thêm dữ liệu thời tiết đã xử lý...
Kích thước sau khi merge lần 3: (3335, 17)


In [8]:
df_form = df_races[['Year', 'RaceName', 'DriverNumber', 'Position', 'Points']].copy()

df_form['AvgPositionLast5'] = df_form.groupby('DriverNumber')['Position'].shift(1).rolling(window=5, min_periods=1).mean()
df_form['AvgPointsLast5'] = df_form.groupby('DriverNumber')['Points'].shift(1).rolling(window=5, min_periods=1).mean()
df_form['PodiumsLast5'] = (df_form.groupby('DriverNumber')['Position'].shift(1) <= 3).rolling(window=5, min_periods=1).sum()

# Lấy các cột feature mới để merge sau này
features_form = df_form[['Year', 'RaceName', 'DriverNumber', 'AvgPositionLast5', 'AvgPointsLast5', 'PodiumsLast5']]

In [9]:
df_quali_analysis = df_qualifying.copy()

# 1. Xác định thời gian phân hạng cuối cùng (Q3 > Q2 > Q1)
df_quali_analysis['FinalQTime'] = df_quali_analysis['Q3'].fillna(df_quali_analysis['Q2']).fillna(df_quali_analysis['Q1'])

# 2. Chuyển đổi thời gian sang giây, xử lý lỗi nếu có
df_quali_analysis['FinalQTime_sec'] = pd.to_timedelta(df_quali_analysis['FinalQTime'], errors='coerce').dt.total_seconds()

# 3. Tìm thời gian pole (thời gian nhanh nhất) cho mỗi cuộc đua
pole_times = df_quali_analysis.groupby(['Year', 'RaceName'])['FinalQTime_sec'].min().reset_index()
pole_times.rename(columns={'FinalQTime_sec': 'PoleTime_sec'}, inplace=True)

# 4. Ghép thông tin thời gian pole trở lại vào bảng chính
df_quali_analysis = pd.merge(df_quali_analysis, pole_times, on=['Year', 'RaceName'], how='left')

# 5. Tính toán feature 'GapToPole'
df_quali_analysis['GapToPole'] = df_quali_analysis['FinalQTime_sec'] - df_quali_analysis['PoleTime_sec']

print("✅ Tạo feature GapToPole thành công!")

# Lấy các cột feature mới để merge sau này
features_quali = df_quali_analysis[['Year', 'RaceName', 'DriverNumber', 'GapToPole']]

df_merged = pd.merge(df_merged, features_form, on=['Year', 'RaceName', 'DriverNumber'], how='left')

df_merged = pd.merge(df_merged, features_quali, on=['Year', 'RaceName', 'DriverNumber'], how='left')


✅ Tạo feature GapToPole thành công!


In [10]:
# Tính giá trị trung bình của GapToPole (bỏ qua các giá trị NaN)
avg_gap_to_pole = df_merged['GapToPole'].mean()

# Điền giá trị trung bình vào những chỗ bị thiếu
df_merged['GapToPole'] = df_merged['GapToPole'].fillna(avg_gap_to_pole)

print("✅ Đã xử lý NaN cho cột GapToPole.")

✅ Đã xử lý NaN cho cột GapToPole.


In [11]:
# --- KIỂM TRA LẠI CODE TRACKTYPE ---

# Code tạo mapping và merge của bạn (đã chính xác)
track_mapping = {
    'Australian Grand Prix': 'Balanced', 'Bahrain Grand Prix': 'Power',
    'Chinese Grand Prix': 'Balanced', 'Azerbaijan Grand Prix': 'Power',
    'Spanish Grand Prix': 'Balanced', 'Monaco Grand Prix': 'Technical',
    'Canadian Grand Prix': 'Power', 'French Grand Prix': 'Balanced',
    'Austrian Grand Prix': 'Power', 'British Grand Prix': 'Balanced',
    'German Grand Prix': 'Balanced', 'Hungarian Grand Prix': 'Technical',
    'Belgian Grand Prix': 'Power', 'Italian Grand Prix': 'Power',
    'Singapore Grand Prix': 'Technical', 'Russian Grand Prix': 'Balanced',
    'Japanese Grand Prix': 'Balanced', 'Mexican Grand Prix': 'Power',
    'United States Grand Prix': 'Balanced', 'Brazilian Grand Prix': 'Balanced',
    'Abu Dhabi Grand Prix': 'Balanced', 'Dutch Grand Prix': 'Technical',
    'Saudi Arabian Grand Prix': 'Power', 'Miami Grand Prix': 'Power',
    'Las Vegas Grand Prix': 'Power',
    'Styrian Grand Prix': 'Power',              # Cùng đường đua với Austrian GP (Red Bull Ring)
    '70th Anniversary Grand Prix': 'Balanced', # Cùng đường đua với British GP (Silverstone)
    'Tuscan Grand Prix': 'Balanced',           # Đường đua Mugello
    'Eifel Grand Prix': 'Balanced',            # Đường đua Nürburgring
    'Portuguese Grand Prix': 'Balanced',       # Đường đua Portimão
    'Emilia Romagna Grand Prix': 'Balanced',   # Đường đua Imola
    'Turkish Grand Prix': 'Balanced',          # Đường đua Istanbul Park
    'Sakhir Grand Prix': 'Power',              # Đường đua Bahrain "Outer Circuit"
    'Mexico City Grand Prix': 'Power',         # Tên mới của Mexican GP
    'São Paulo Grand Prix': 'Balanced',        # Tên mới của Brazilian GP
    'Qatar Grand Prix': 'Balanced'
}
df_track_type = pd.DataFrame(list(track_mapping.items()), columns=['RaceName', 'TrackType'])
df_track_type_encoded = pd.get_dummies(df_track_type, columns=['TrackType'], prefix='Track')

df_merged = pd.merge(df_merged, df_track_type_encoded, on='RaceName', how='left')

print("✅ Chuyển đổi feature 'TrackType' thành các cột số thành công.")

✅ Chuyển đổi feature 'TrackType' thành các cột số thành công.


In [12]:
df_track_history = df_races[['Year', 'RaceName', 'DriverNumber', 'Position']].copy()

# Tính vị trí trung bình lịch sử
avg_pos_series = df_track_history.groupby(['RaceName', 'DriverNumber'])['Position'].apply(
    lambda x: x.shift(1).expanding(min_periods=1).mean()
)

# Tính tổng số podium lịch sử
podiums_series = df_track_history.groupby(['RaceName', 'DriverNumber'])['Position'].apply(
    lambda x: (x.shift(1) <= 3).expanding(min_periods=1).sum()
)
# Gán kết quả trở lại DataFrame
# Chúng ta cần reset_index để có thể gán lại một cách chính xác
df_track_history['AvgPositionAtTrack'] = avg_pos_series.reset_index(level=[0,1], drop=True)
df_track_history['PodiumsAtTrack'] = podiums_series.reset_index(level=[0,1], drop=True)


print("✅ Tạo feature thành tích lịch sử thành công!")

# --- Phần hợp nhất và xử lý NaN (giữ nguyên vì logic đã đúng) ---
features_track_history = df_track_history[['Year', 'RaceName', 'DriverNumber', 'AvgPositionAtTrack', 'PodiumsAtTrack']]
df_merged = pd.merge(df_merged, features_track_history, on=['Year', 'RaceName', 'DriverNumber'], how='left')

avg_pos_fill_value = df_merged['FinalPosition'].mean()
df_merged['AvgPositionAtTrack'] = df_merged['AvgPositionAtTrack'].fillna(avg_pos_fill_value)
df_merged['PodiumsAtTrack'] = df_merged['PodiumsAtTrack'].fillna(0)

print("✅ Hợp nhất và xử lý NaN thành công!")

✅ Tạo feature thành tích lịch sử thành công!
✅ Hợp nhất và xử lý NaN thành công!


In [13]:
cols_to_fill_zero = [
    'AvgPointsLast5',
    'AvgPositionLast5'
]
for col in cols_to_fill_zero:
    df_merged[col] = df_merged[col].fillna(0)

In [14]:

output_filename = '../data/processed/final_master_df.csv'

# Sử dụng .to_csv() để lưu DataFrame
# index=False là rất quan trọng để không lưu cột chỉ mục (0, 1, 2...) của pandas vào file
df_merged.to_csv(output_filename, index=False)

# In ra một thông báo xác nhận để bạn biết quá trình đã hoàn tất
print(f"🎉 DataFrame cuối cùng đã được lưu thành công vào file '{output_filename}'!")

🎉 DataFrame cuối cùng đã được lưu thành công vào file '../data/processed/final_master_df.csv'!
